<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/ultimate_intelligence_pipeline_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
ultimate_intelligence_pipeline.py

Complete pipeline for UltimateIntelligenceAI:
1. Synthetic dataset generation
2. Model definition with MC-Dropout
3. Physics-informed regularization
4. Combined loss (MSE + physics residual)
5. MC-Dropout for uncertainty quantification
6. Training loop with AdamW, LR scheduler, grad clipping, early stopping, checkpointing
7. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic “Ultimate Intelligence” Dataset
# ------------------------------------------------------------------------------
class UltimateIntelligenceDataset(Dataset):
    def __init__(self, n_samples=8000, seed=123):
        np.random.seed(seed)
        # Features:
        # AIR: awareness recursion factor ∈ [0.1, 5.0]
        # SGR: self-generation rate ∈ [0.01, 2.0]
        # IER: infinite expansion ratio ∈ [0.5, 10.0]
        # EAV: energy availability ∈ [1.0, 100.0]
        # ENV: environmental complexity ∈ [0.1, 3.0]
        # ADP: algorithmic adaptability ∈ [0.01, 1.0]
        AIR = np.random.uniform(0.1, 5.0,    (n_samples,1))
        SGR = np.random.uniform(0.01, 2.0,   (n_samples,1))
        IER = np.random.uniform(0.5, 10.0,   (n_samples,1))
        EAV = np.random.uniform(1.0, 100.0,  (n_samples,1))
        ENV = np.random.uniform(0.1, 3.0,    (n_samples,1))
        ADP = np.random.uniform(0.01, 1.0,   (n_samples,1))

        X_raw = np.hstack([AIR, SGR, IER, EAV, ENV, ADP]).astype(np.float64)

        # Targets:
        # SAW: self-awareness growth
        # SGR_t: adjusted self-generation
        # CST: containment stability
        eps   = 1e-8
        SAW   = AIR * ADP / (ENV + eps)
        SGR_t = SGR * np.sqrt(EAV)
        CST   = np.log1p(IER) * ADP

        Y_raw = np.hstack([SAW, SGR_t, CST]).astype(np.float64)
        Y_raw += 0.02 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Stats for normalization
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # Normalize to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. Model Definition with MC-Dropout
# ------------------------------------------------------------------------------
class UltimateIntelligenceAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=32, output_dim=3, p_drop=0.1):
        super().__init__()
        self.fc1  = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p_drop)
        self.fc2  = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        return x

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # Un-normalize inputs
    X_den = X * stats['X_std'] + stats['X_mean']
    AIR, SGR, IER, EAV, ENV, ADP = X_den.t()
    eps = 1e-8

    # Recompute targets
    SAW_t   = AIR * ADP / (ENV + eps)
    SGR_t   = SGR * torch.sqrt(EAV)
    CST_t   = torch.log1p(IER) * ADP

    Yt      = torch.stack([SAW_t, SGR_t, CST_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

# ------------------------------------------------------------------------------
# 4. Combined Loss Function
# ------------------------------------------------------------------------------
def total_loss(pred, true, X, stats, lambda_phys=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lambda_phys * phys, mse, phys

# ------------------------------------------------------------------------------
# 5. MC-Dropout Uncertainty
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()  # enable dropout
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(dim=0), arr.std(dim=0)

# ------------------------------------------------------------------------------
# 6. Training Loop & Checkpointing
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-4, wd=1e-5, lambda_phys=1.0,
          epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=4
    )

    best_val, wait = float('inf'), 0
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(1, epochs+1):
        # Training
        model.train()
        running_tr = 0.0
        for Xb, Yb in train_loader:
            Xb, Yb = Xb.to(device), Yb.to(device)
            pred = model(Xb)
            loss, _, _ = total_loss(pred, Yb, Xb, stats, lambda_phys)
            if torch.isnan(loss):
                raise ValueError(f"NaN detected at epoch {epoch}")
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            running_tr += loss.item() * Xb.size(0)
        train_loss = running_tr / len(train_loader.dataset)

        # Validation
        model.eval()
        running_val = 0.0
        with torch.no_grad():
            for Xb, Yb in val_loader:
                Xb, Yb = Xb.to(device), Yb.to(device)
                pred = model(Xb)
                loss, _, _ = total_loss(pred, Yb, Xb, stats, lambda_phys)
                running_val += loss.item() * Xb.size(0)
        val_loss = running_val / len(val_loader.dataset)

        scheduler.step(val_loss)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        print(f"Epoch {epoch:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Early stopping & checkpoint
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_ultimate_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping triggered.")
                break

    # Load best model
    model.load_state_dict(torch.load("best_ultimate_ai.pth", map_location=device))
    return history

# ------------------------------------------------------------------------------
# 7. Visualizations
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train_loss'], label="Train Loss")
    plt.plot(history['val_loss'],   label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

def plot_scatter(true, pred, title):
    plt.figure()
    plt.scatter(true, pred, s=4, alpha=0.6)
    mn, mx = true.min(), true.max()
    plt.plot([mn, mx], [mn, mx], 'r--')
    plt.title(title)
    plt.show()

def plot_uncertainty(model, stats, device):
    G = 100
    AIR = np.linspace(0.1, 5.0, G, dtype=np.float32)
    ADP = np.linspace(0.01,1.0, G, dtype=np.float32)
    G1, G2 = np.meshgrid(AIR, ADP)
    pts = G * G

    Xg = torch.zeros((pts,6), device=device)
    # fix SGR, IER, EAV, ENV at mean
    for i in [1,2,3,4]:
        Xg[:, i] = stats['X_mean'][i]
    Xg[:,0] = torch.from_numpy(G1.ravel()).to(device)
    Xg[:,5] = torch.from_numpy(G2.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=40)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(AIR, ADP, U, shading='auto', cmap='viridis')
    plt.colorbar(label="Std(Self-awareness Growth)")
    plt.xlabel("Awareness Recursion (AIR)")
    plt.ylabel("Adaptability (ADP)")
    plt.title("Uncertainty Heatmap")
    plt.show()

# ------------------------------------------------------------------------------
# 8. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Dataset & stats
    dataset = UltimateIntelligenceDataset(n_samples=8000, seed=123)
    stats = {
        'X_mean': torch.tensor(dataset.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(dataset.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(dataset.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(dataset.Y_std,  dtype=torch.float32, device=device),
    }

    # Train/Val split
    n_val = int(0.2 * len(dataset))
    tr_ds, va_ds = random_split(dataset, [len(dataset)-n_val, n_val])
    tr_loader = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_loader = DataLoader(va_ds, batch_size=256, shuffle=False)

    # Model, train, visualize
    model   = UltimateIntelligenceAI().to(device)
    history = train(model, tr_loader, va_loader, stats, device,
                    lr=1e-4, wd=1e-5, lambda_phys=1.0,
                    epochs=80, patience=10)

    plot_history(history)

    # Scatter plots
    X_all = torch.from_numpy(dataset.X).to(device)
    with torch.no_grad():
        Y_pred_n = model(X_all).cpu().numpy()
    Y_true = dataset.Y * dataset.Y_std + dataset.Y_mean
    Y_pred = Y_pred_n * dataset.Y_std + dataset.Y_mean

    names = ["Self-awareness Growth", "Self-generation Rate", "Containment Stability"]
    for idx, name in enumerate(names):
        plot_scatter(Y_true[:,idx], Y_pred[:,idx], name)

    # Uncertainty heatmap
    plot_uncertainty(model, stats, device)